In [3]:
# import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utility
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import train_test_split


In [4]:
import pandas as pd
import numpy as np


In [5]:
train_df = pd.read_pickle(r'Yelp-4/training_df.pkl')    
vali_df = pd.read_pickle(r'Yelp-4/valiing_df.pkl')       # for validation
testing_df = pd.read_pickle(r'Yelp-4/testing_df.pkl')     # for testing
key_genre = pd.read_pickle(r'Yelp-4/key_genre.pkl')  
item_idd_genre_list = pd.read_pickle(r'Yelp-4/item_idd_genre_list.pkl')   
genre_item_vector = pd.read_pickle(r'Yelp-4/genre_item_vector.pkl')    
genre_count = pd.read_pickle(r'Yelp-4/genre_count.pkl')      
user_genre_count = pd.read_pickle(r'Yelp-4/user_genre_count.pkl') 

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)

item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

In [6]:
item_genre = np.zeros((num_item, num_genre))
for i in range(num_item):
    gl = item_genre_list[i]
    for k in range(num_genre):
        if key_genre[k] in gl:
            item_genre[i, k] = 1.0

In [7]:
item_genre

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 1., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [8]:
# train_df = train_df[train_df['user_id'].isin(vali_df['user_id'].unique())]
# train_df = train_df[train_df['item_id'].isin(vali_df['item_id'].unique())]

In [9]:
def create_user_list(df, user_size):
    user_list = [list() for u in range(user_size)]
    for row in df.itertuples():
        user_list[row.user_id].append(row.item_id)
    return user_list

def create_pair(user_list):
    pair = []
    for user, item_list in enumerate(user_list):
        pair.extend([(user, item) for item in item_list])
    return pair

In [10]:
train_df_loop = pd.read_pickle(r'Yelp-4/training_df.pkl')  

In [11]:
user_train_list=[]
user_tune_list=[]
for user in train_df['user_id'].unique():
    train_idx = train_df_loop[train_df_loop['user_id'] == user].sample(n=1, random_state=18).index[0]
    user_train_list.append(train_idx)
    train_df_loop = train_df_loop.drop(train_idx)
    tune_idx = train_df_loop[train_df_loop['user_id'] == user].sample(n=1, random_state=181).index[0]
    user_tune_list.append(tune_idx)
    train_df_loop = train_df_loop.drop(tune_idx)

In [12]:
train_df_loop = pd.read_pickle(r'Yelp-4/training_df.pkl')
tune_df_loop = pd.read_pickle(r'Yelp-4/training_df.pkl')  
train_df = pd.read_pickle(r'Yelp-4/training_df.pkl') 

In [13]:
train_df.loc[train_df.index[user_tune_list]]['user_id'].nunique(),train_df.loc[train_df.index[user_train_list]]['user_id'].nunique()

(8263, 8263)

In [14]:
indexes_to_keep_train_item = set(range(train_df.shape[0])) - set(user_tune_list)
train_df_loop = train_df_loop.take(list(indexes_to_keep_train_item))
indexes_to_keep_tune_item = set(range(train_df.shape[0])) - set(user_train_list)
tune_df_loop = tune_df_loop.take(list(indexes_to_keep_tune_item))

In [15]:
item_train_list=[]
item_tune_list=[]
for item in train_df['item_id'].unique():
    train_idx = train_df_loop[train_df_loop['item_id'] == item].sample(n=1, random_state=18).index[0]
    item_train_list.append(train_idx)
    train_df_loop = train_df_loop.drop(train_idx)
    if train_idx in tune_df_loop.index:
        tune_df_loop = tune_df_loop.drop(train_idx)
    else:
        tune_df_loop = tune_df_loop
    if len(tune_df_loop[tune_df_loop['item_id'] == item]) != 0:
        tune_idx = tune_df_loop[tune_df_loop['item_id'] == item].sample(n=1, random_state=18).index[0]
    
        item_tune_list.append(tune_idx)
        tune_df_loop = tune_df_loop.drop(tune_idx)
        if tune_idx in train_df_loop.index:
            train_df_loop = train_df_loop.drop(tune_idx)
        else:
            train_df_loop = train_df_loop
    else:
        continue

In [16]:
# train_df_loop = pd.read_pickle(r'Yelp-2/training_df.pkl')  
train_df = pd.read_pickle(r'Yelp-4/training_df.pkl') 

In [17]:
train_df.loc[train_df.index[item_train_list]]['item_id'].nunique(),train_df.loc[train_df.index[item_tune_list]]['item_id'].nunique()

(4420, 4420)

In [18]:
num_item, num_user

(4420, 8263)

In [19]:
train_list_preocc = train_df.loc[train_df.index[list(set(user_train_list+item_train_list))]]

In [20]:
len(train_list_preocc)

12409

In [21]:
tune_list_preocc = train_df.loc[train_df.index[list(set(user_tune_list+item_tune_list))]]

In [22]:
tune_list_preocc

,item_id,user_id,r
0,0,0,True
131073,276,7213,True
8,11,0,True
32776,93,1907,True
32777,2949,1907,True
...,...,...,...
65524,652,3655,True
32758,3477,1905,True
98294,961,5337,True
98297,276,5451,True


In [26]:
tune_list_preocc.groupby('item_id').size()  

item_id
0        5
1       27
2       20
3        1
4        2
        ..
4415     1
4416     1
4417     1
4418     1
4419     1
Length: 4420, dtype: int64

In [25]:
import matplotlib.pyplot as plt

In [ ]:
with open("Yelp-4/tune_df_mininum.pkl", "wb") as f:
    pickle.dump(tune_set, f, pickle.HIGHEST_PROTOCOL)

In [335]:
len(tune_list_preocc)

12366

In [336]:
train_df.shape[0]

150616

In [337]:
indexes_to_keep = set(range(train_df.shape[0])) - set(user_train_list+item_train_list+user_tune_list+item_tune_list)
sample_pool = train_df.take(list(indexes_to_keep))

In [338]:
len(train_df)

150616

In [339]:
len(sample_pool)+len(train_list_preocc)+len(tune_list_preocc)

150616

In [340]:
test_ratio = (int(len(train_df)*1/6)-len(tune_list_preocc))/len(sample_pool)
test_ratio

0.10120707877400847

In [341]:
train_set, tune_set = train_test_split(sample_pool,test_size = test_ratio, random_state = 181)

In [342]:
train_set = pd.concat([train_set,train_list_preocc])
tune_set = pd.concat([tune_set,tune_list_preocc])

In [343]:
len(tune_set)/len(train_set)

0.19999362620902847

In [344]:
train_set['user_id'].nunique(),train_set['item_id'].nunique()

(8263, 4420)

In [345]:
tune_set['user_id'].nunique(),tune_set['item_id'].nunique()

(8263, 4420)

In [346]:
with open("Yelp-4/train_df.pkl", "wb") as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
with open("Yelp-4/tune_df.pkl", "wb") as f:
    pickle.dump(tune_set, f, pickle.HIGHEST_PROTOCOL)

In [113]:
train_user_list = create_user_list(train_set, num_user)
test_user_list = create_user_list(testing_df, num_user)
train_pair = create_pair(train_user_list)


In [114]:
num_sample, user_list, item_pos_list, item_neg_list = utility.negative_sample(testing_df, num_user,
                                                                                      num_item, 5)

In [115]:
pair = []
pair.extend([(int(user), int(item)) for user, item in zip(user_list,item_pos_list)])

In [116]:
len(pair)

101625

In [117]:
len(train_pair)

10163

In [118]:
dataset = {'user_size': num_user, 'item_size': num_item,
            'train_user_list': train_user_list, 'test_user_list': test_user_list,
            'train_pair': pair}

In [119]:
dirname = os.path.dirname(os.path.abspath('preprocessed/yelp-4-tune-data.pickle'))
os.makedirs(dirname, exist_ok=True)
with open('preprocessed/yelp-4-tune-data', 'wb') as f:
    pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)